In [1]:
import numpy as np
from copy import deepcopy as deep_copy
import pandas as pd
import csv
from pgmpy.estimators import BicScore
from pgmpy.models import BayesianModel
import networkx as nx

In [42]:
def classic_pso(nvar, ncal, type, function, chi, w):

    class Problem:

        def __init__(self, nvar, ncal, type, function, chi, w):
            #
            # particle parameters
            self.c1 = 1.4
            self.c2 = 1.4
            self.chi = chi
            self.w = w
            #
            # problem parameters
            self.dimension = nvar
            self.n_cal = ncal
            self.lower_bound = -0.49
            self.upper_bound = 2.49
            self.type_PSO = type         # (Global / Local)
            self.function = function     # (Sphere / Rastrigin)
            #
            # population parameters
            self.nao_dag = []
            self.g_best_position = []
            self.g_best_value = np.inf   # default
            if np.sqrt(self.n_cal) > 200:
                self.swarm_size = 200
            else:
                self.swarm_size = int(np.sqrt(self.n_cal))
            self.n_iter = self.n_cal//self.swarm_size
            if self.type_PSO == 'Global':
                self.neighbours = self.swarm_size
            elif self.type_PSO == 'Local':
                self.neighbours = 2
            #
            # Read Cancer problem data ---------------------------------------------------------------------------------
            if self.function == 'Cancer':
                with open('cancer100k.csv') as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    aux = 0
                    data = []
                    data1 = [[] for i in range(5)]
                    for row in csv_reader:
                        data.append(row)
                        for i in range(len(row)):
                            data1[i].append(row[i])
                        aux = aux + 1
                        if aux == 50001:
                            break
                    data = {}
                for i in range(len(data1)):
                    data[data1[i][0]] = [data1[i][j] for j in range(1, len(data1[i]))]
                data = pd.DataFrame(data)
                #print("Data: ")
                #print(data)  # Dada retieved from file
                self.data = data
                self.bic_score=BicScore(data)
                self.nodes = ['Pollution', 'Smoker', 'Cancer', 'Xray', 'Dyspnoea']
                
        def vetor_Rede(self, solucao, nodes):
            G_aux = BayesianModel()
            # G_aux.add_nodes_from(nodes)
            k = 0
            aux = 1
            for i in range(1, len(nodes)):
                for j in range(aux):
                    if solucao[k] == 1:
                        if nodes[i] in G_aux.nodes() and nodes[j] in G_aux.nodes() and nx.has_path(G_aux,
                                                                                                   nodes[j],
                                                                                                   nodes[i]):
                            return False
                        else:
                            G_aux.add_edge(nodes[i], nodes[j])
                    elif solucao[k] == 2:
                        if nodes[i] in G_aux.nodes() and nodes[j] in G_aux.nodes() and nx.has_path(G_aux,
                                                                                                   nodes[i],
                                                                                                   nodes[j]):
                            return False
                        else:
                            G_aux.add_edge(nodes[j], nodes[i])
                    k = k + 1
                aux = aux + 1
            for i in nodes:
                if i not in G_aux.nodes():
                    return False
            return G_aux

        def cost_function(self, x):
            #
            # ============= Cancer Bayes Model =========================================================================
            if self.function == 'Cancer':
                G = self.vetor_Rede(solucao=x, nodes=self.nodes)
                return BicScore(self.data).score(G)
            #
            # ============= Asia Bayes Model ===========================================================================
            # todo
            # ==========================================================================================================
            

    class Particle(Problem):

        def __init__(self):
            Problem.__init__(self, nvar, ncal, type, function, chi, w)
            flag = False
            while flag == False:
                aux_2 = np.random.rand(self.dimension) * \
                    (self.upper_bound - self.lower_bound) + self.lower_bound
                aux = []
                for i in aux_2:
                    aux.append(int(i.round()))
                if aux not in self.nao_dag:
                #if flag2 == False:
                    G=self.vetor_Rede(aux, self.nodes)
                    if G:
                        self.position = aux
                        self.fitness = Problem.cost_function(self, x=self.position)
                        flag = True
                        #fitness.append(abs(bic_score.score(G)))
                    else:
                        self.nao_dag.append(aux)     

        def calc_fitness(self):
            self.fitness = Problem.cost_function(self, x=list(self.position.round()))
            return self.fitness

        def move(self, velocity):
            self.position = self.position + velocity
            for i in range(self.dimension):
                flag = False
                while flag is False:    # make the particle bounce
                    if self.position[i] > self.upper_bound:
                        self.position[i] = self.upper_bound - (self.position[i] - self.upper_bound)
                    if self.position[i] < self.lower_bound:
                        self.position[i] = self.lower_bound - (self.position[i] - self.lower_bound)
                    if self.lower_bound < self.position[i] < self.upper_bound:
                        flag = True
            return self.position

        def update_velocity(self, velocity, p_best_position, g_best_position):
            r1 = np.random.rand(self.dimension)
            r2 = np.random.rand(self.dimension)
            cognitive = self.c1 * r1 * (p_best_position - self.position)
            social = self.c2 * r2 * (g_best_position - self.position)
            velocity = self.chi * (self.w * velocity + cognitive + social)
            return velocity

    class PSO(Problem):

        def __init__(self):
            Problem.__init__(self, nvar, ncal, type, function, chi, w)
            self.swarm = np.zeros([self.swarm_size, self.dimension])
            self.fitness = np.zeros(self.swarm_size) + np.inf
            self.velocity = np.zeros([self.swarm_size, self.dimension])
            self.personal_best = np.zeros(self.swarm_size) + np.inf
            self.personal_best_pos = np.zeros([self.swarm_size, self.dimension])
            if self.type_PSO == 'Global':
                self.global_best = np.inf
                self.global_best_pos = np.zeros(self.dimension)
            elif self.type_PSO == 'Local':
                self.global_best = np.zeros(self.swarm_size) + np.inf
                self.global_best_pos = np.zeros([self.swarm_size, self.dimension])
            #
            # initialize swarm
            for i in range(self.swarm_size):
                part = Particle()
                self.swarm[i, :] = part.position
                self.fitness[i] = part.fitness
                self.personal_best_pos[i, :] = self.swarm[i, :]
                self.personal_best[i] = self.fitness[i]
            self.global_best, self.global_best_pos = \
                self.update_best(g_best_value=self.global_best, g_best_position=self.global_best_pos)
            self.optimize()

        def optimize(self):
            for i in range(self.n_iter):
                print('iteration = ' + str(i))
                # Update position of the particles
                for k in range(self.swarm_size):
                    part = Particle()
                    part.position = self.swarm[k, :]
                    if self.type_PSO == 'Global':
                        self.velocity[k, :] = part.update_velocity(velocity=self.velocity[k, :],
                                                                   p_best_position=self.personal_best_pos[k, :],
                                                                   g_best_position=self.global_best_pos)
                    elif self.type_PSO == 'Local':
                        self.velocity[k, :] = part.update_velocity(velocity=self.velocity[k, :],
                                                                   p_best_position=self.personal_best_pos[k, :],
                                                                   g_best_position=self.global_best_pos[k, :])
                    part.position = part.move(self.velocity[k, :])
                    #
                    # nao_dag restriction
                    aux = list(part.position.round())
                    if aux not in self.nao_dag:
                        G=self.vetor_Rede(aux, self.nodes)
                        if G:
                            self.swarm[k, :] = part.position
                            self.fitness[k] = part.calc_fitness()
                            flag = True
                        else:
                            self.nao_dag.append(aux)     
                            self.swarm[k, :] = part.position
                    if self.fitness[k] < self.personal_best[k]:
                        self.personal_best[k] = self.fitness[k]
                        self.personal_best_pos[k, :] = self.swarm[k, :]
                self.global_best, self.global_best_pos = \
                    self.update_best(g_best_value=self.global_best, g_best_position=self.global_best_pos)
            return self.global_best, self.global_best_pos

        def update_best(self, g_best_value, g_best_position):
            if self.type_PSO == 'Global':
                for i in range(self.swarm_size):
                    if self.fitness[i] < g_best_value:
                        g_best_value = deep_copy(self.fitness[i])
                        g_best_position = deep_copy(self.swarm[i, :])
            elif self.type_PSO == 'Local':
                aux_g_best_value = deep_copy(g_best_value)
                aux_1 = -1
                aux_2 = 1
                for i in range(self.swarm_size):
                    if aux_2 >= self.swarm_size:
                        aux_2 = 0
                    vector = [aux_g_best_value[aux_1], self.fitness[i], aux_g_best_value[aux_2]]
                    g_best_value[i] = deep_copy(np.min(vector))
                    if np.argmin(vector) == 0:
                        g_best_position[i, :] = deep_copy(g_best_position[aux_1, :])
                    elif np.argmin(vector) == 1:
                        g_best_position[i, :] = deep_copy(self.swarm[i, :])
                    elif np.argmin(vector) == 2:
                        g_best_position[i, :] = deep_copy(g_best_position[aux_2, :])
                    aux_1 += 1
                    aux_2 += 1
            return g_best_value, g_best_position

    prob = Problem(nvar, ncal, type, function, chi, w)

    p = PSO()

    print('========== Parameters of PSO ===========')
    print('dimension = ' + str(prob.dimension))
    print('n_cal = ' + str(prob.n_cal))
    print('pop size = ' + str(prob.swarm_size))
    print('chi = ' + str(prob.chi))
    print('weight inertia = ' + str(prob.w))
    print('type_PSO = ' + prob.type_PSO)
    print('function = ' + str(prob.function))
    print('================ BEST ==================')
    if prob.type_PSO == 'Global':
        print('global best =' + str(p.global_best))
        print('and its position = ' + str(p.global_best_pos))
        gbest = p.global_best
        gbest_position = p.global_best_pos
    elif prob.type_PSO == 'Local':
        print('global best =' + str(np.min(p.global_best)))
        print('and its position = ' + str(p.global_best_pos[np.argmin(p.global_best)]))
        gbest = np.min(p.global_best)
        gbest_position = p.global_best_pos[np.argmin(p.global_best)]
    return gbest, gbest_position


In [43]:
f, x = classic_pso(nvar=10, ncal=100, type='Global', function='Cancer', chi=0.9, w=0.9)

[2.27295942 2.         0.         1.35426573 0.2091309  0.
 1.86030076 0.69527773 0.         1.        ]
[2.0, 2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 1.0]
[1.82981484 2.28375341 0.         1.53058499 0.65746462 0.
 1.         1.60526672 0.50499511 0.87349445]
[2.0, 2.0, 0.0, 2.0, 1.0, 0.0, 1.0, 2.0, 1.0, 1.0]
[2.         1.32889892 0.37291388 0.97957005 0.         0.
 0.21899982 1.39969503 0.88129712 1.58584138]
[2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0]
[2.         2.08779948 0.         2.         1.29138437 0.00352614
 0.6989262  1.         0.86974604 1.        ]
[2.0, 2.0, 0.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
[0.25217367 2.         0.90870572 2.         0.         0.63304076
 0.64328271 0.02086425 0.87577062 1.        ]
[0.0, 2.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0]
[2.         2.         0.         2.         1.8255456  0.
 1.7322941  1.16587066 0.         0.7769594 ]
[2.0, 2.0, 0.0, 2.0, 2.0, 0.0, 2.0, 1.0, 0.0, 1.0]
[ 2.          0.615894    0.44679781  2.08887753

[2.0, 1.0, -0.0, 1.0, 0.0, 2.0, 0.0, 1.0, -0.0, 1.0]
[ 1.74281979  1.44838938 -0.40840682  1.35451598  0.27518952 -0.17956304
  1.09494004  0.99097836 -0.10897069  1.04869992]
[2.0, 1.0, -0.0, 1.0, 0.0, -0.0, 1.0, 1.0, -0.0, 1.0]
[ 2.          0.85822589 -0.01304065  1.52017908  2.15131304 -0.39286727
  0.80854117  2.35292462 -0.30511061  0.55293448]
[2.0, 1.0, -0.0, 2.0, 2.0, -0.0, 1.0, 2.0, -0.0, 1.0]
[ 2.          1.83819146  0.09253342  2.14305159  0.60659959 -0.35886987
  1.06102477  1.          0.66672911  0.96639059]
[2.0, 2.0, 0.0, 2.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
[ 2.18945513  0.89882822 -0.02080174  1.1825736  -0.1754044   0.47639581
  0.80545656  1.         -0.01374897  1.46326585]
[2.0, 1.0, -0.0, 1.0, -0.0, 0.0, 1.0, 1.0, -0.0, 1.0]
[ 2.          1.85261945 -0.22923857  1.42823495  0.22285539  1.23769915
  0.94787306  1.         -0.45627166  1.04591917]
[2.0, 2.0, -0.0, 1.0, 0.0, 1.0, 1.0, 1.0, -0.0, 1.0]
[ 2.          1.3184772  -0.00831567  1.06396423  0.40926883 -0.2

  0.80586605  1.23315305 -0.46472371  1.56544371]
[2.0, 1.0, -0.0, 2.0, 0.0, -0.0, 1.0, 1.0, -0.0, 2.0]
[ 1.70876615  0.89226838 -0.07674304  1.53875159  0.30668936  0.61931774
  0.93035707  1.19509642  0.12821292  1.55347535]
[2.0, 1.0, -0.0, 2.0, 0.0, 1.0, 1.0, 1.0, 0.0, 2.0]
[ 1.77956447  1.46445323  0.36843938  1.18181771  0.639603   -0.21437168
  0.8766598   1.3722508   0.61596461  1.46862324]
[2.0, 1.0, 0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
[ 1.70352255  0.88126494 -0.03745234  1.16818778  0.3622187  -0.28083176
  0.86200789  1.15236808 -0.48688777  1.2484673 ]
[2.0, 1.0, -0.0, 1.0, 0.0, -0.0, 1.0, 1.0, -0.0, 1.0]
[ 1.95405718  1.51804665  0.55106835  1.4388787   0.02289056 -0.08965425
  1.59156898  1.35495102 -0.15598173  1.79148719]
[2.0, 2.0, 1.0, 1.0, 0.0, -0.0, 2.0, 1.0, -0.0, 2.0]
[ 1.81886344  2.37975998 -0.06075608  1.89104621  0.50844445 -0.41337063
  0.74567625  1.18025545 -0.11692781  1.70863639]
[2.0, 2.0, -0.0, 2.0, 1.0, -0.0, 1.0, 1.0, -0.0, 2.0]
[ 1.71654499  1.

  0.94665126  1.00917494  0.04344147  1.39005733]
[2.0, 1.0, -0.0, 1.0, 0.0, -0.0, 1.0, 1.0, 0.0, 1.0]
========== Parameters of PSO ===========
dimension = 10
n_cal = 100
pop size = 10
chi = 0.9
weight inertia = 0.9
type_PSO = Global
function = Cancer
================ BEST ==================
global best =-106563.00931138193
and its position = [ 1.7105564   0.90620121 -0.06185073  1.47071563  0.43570018 -0.39075858
  0.88031549  1.23368573 -0.13637495  1.67415442]


[1.22920521 0.30967815 0.50890216 0.67520767 1.91179698]
[1.2292052051782614, 0.30967815072264315, 0.5089021555246784, 0.6752076656365353, 1.911796981270381]
